<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#O-que-veremos?" data-toc-modified-id="O-que-veremos?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>O que veremos?</a></span></li><li><span><a href="#Requisitos-Básicos" data-toc-modified-id="Requisitos-Básicos-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Requisitos Básicos</a></span><ul class="toc-item"><li><span><a href="#Teoria" data-toc-modified-id="Teoria-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Teoria</a></span></li><li><span><a href="#Técnico" data-toc-modified-id="Técnico-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Técnico</a></span></li></ul></li><li><span><a href="#Criando-Pipelines" data-toc-modified-id="Criando-Pipelines-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Criando Pipelines</a></span><ul class="toc-item"><li><span><a href="#Sem-Pipeline" data-toc-modified-id="Sem-Pipeline-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Sem Pipeline</a></span></li><li><span><a href="#Com-Pipeline" data-toc-modified-id="Com-Pipeline-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Com Pipeline</a></span></li></ul></li><li><span><a href="#Dissecando-com_pipeline.py" data-toc-modified-id="Dissecando-com_pipeline.py-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Dissecando com_pipeline.py</a></span></li></ul></div>

# O que veremos? 

Em 99% dos modelos de Machine Learning criados(carece de fontes), nós teremos que pré-processar e transformar os dados antes deles serem passados para um estimador do *scikit-learn*.<br>

O problema disso é que nós teremos muitas linhas de código com objetos que possuem os mesmos métodos .fit() e .transform() e um final com .fit() e .predict(). Nós aprenderemos a colocar todos esses objetos dentro um único PIPELINE, assim usaremos apenas o PIPELINE.

# Requisitos Básicos 

## Teoria
Não há nenhuma teoria básica que seja condição necessária para que você consiga seguir com esse notebook.

## Técnico 
- Python | Requisitos em **02-Python-Packages-Libraries/0-Python**
    - Básico de Orientação a Objetos. Somente saber o que são objetos, classes, métodos, atributos etc. Nomenclatura e terminologia básicas
    - Imports de bibliotecas
- NumPy | Requisitos em **02-Python-Packages-Libraries/01-NumPy**
    - Arrays
    - Manipulação básica de arrays
- Pandas | Requisitos em **02-Python-Packages-Libraries/02-Pandas**
    - Leitura de dados
    - Dataframes e Series
    - Manipulação básica dos dois acima
- Scikit-Learn | Requisitos em **02-Python-Packages-Libraries/03-Scikit-Learn**
    - Transformadores e pré-processadores
    - 

# Criando Pipelines 

## Sem Pipeline

Primeiro, vamos fazer um modelo sem usar o Pipeline e ver como o código ficaria.<br>

Vamos usar o dataset [Titanic do Kaggle](https://www.kaggle.com/c/titanic), entre no link para saber mais sobre a competição no próprio [Kaggle](https://www.kaggle.com/).<br>

Alguns pontos:
- A coluna PassengerId será usada como index
- Usaremos apenas as features Age e Fare, as duas serão nossa Design Matrix X
    - Age tem dados faltantes
- Survived é o nosso Target Values y
- Não iremos separar os dados em treino e teste. Nunca faça isso, aqui é somente um exemplo para mostrarmos o funcionamento geral do Pipeline, depois iremos mostrar com profundidade tudo o que temos.

In [4]:
"""
------sem_pipeline.py------

Script que mostra duas transformacoes:
    - Inserimos a media onde temos valores faltantes
    - Fazer o StandardScaler dos dados apos inserir os dados faltantes
Apos as duas transformacoes, nos usamos o estimador LogisticRegression
Repare na quantidade de passos que devemos fazer.
"""
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

PATH_FILE = 'data/titanic.csv'
USE_COLS = ['PassengerId', 'Survived', 'Age', 'Fare']

df_titanic = pd.read_csv(PATH_FILE, usecols=USE_COLS, index_col='PassengerId')

X = df_titanic.drop(columns='Survived')
y = df_titanic.loc[:, 'Survived']

print('Porcentagem de dados faltantes nas features')
print(f'{(X.isna().mean()*100).round(2)}')
print('Veja que a feature Age tem ~20% de dados faltantes')
print('--------------------------------------------------')

imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()

# inserindo a media em dados faltantes
# vamos usar primeiro .fit() e depois fit_transfor()
imputer.fit(X)
X_t_impute = imputer.transform(X)

# fazendo o scaling dos dados com StandardScaler
# apos inserirmos dados vazios
scaler.fit(X_t_impute)
X_t_scaler = scaler.transform(X_t_impute)

log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_t_scaler, y)

# vamos ver a acuracia do modelo
# estamos fazendo isso nos mesmos 
# dados que treinamos, nunca faca isso
acc = log_reg.score(X_t_scaler, y)*100
print(f'\nAcc nos mesmos dados que treinamos: {acc:.2f}%')

Porcentagem de dados faltantes nas features
Age     19.87
Fare     0.00
dtype: float64
Veja que a feature Age tem ~20% de dados faltantes
--------------------------------------------------

Acc nos mesmos dados que treinamos: 65.43%


Veja a saída de cada transformação

In [5]:
# Design Matrix Original
X

,Age,Fare
PassengerId,,
1,22.0,7.2500
2,38.0,71.2833
3,26.0,7.9250
4,35.0,53.1000
5,35.0,8.0500
...,...,...
887,27.0,13.0000
888,19.0,30.0000
889,NaN,23.4500


In [6]:
# Design Matrix apos inserir a media
# onde tinhamos dados faltantes
X_t_impute

array([[22.        ,  7.25      ],
       [38.        , 71.2833    ],
       [26.        ,  7.925     ],
       ...,
       [29.69911765, 23.45      ],
       [26.        , 30.        ],
       [32.        ,  7.75      ]])

In [7]:
# Design Matrix apos inserir a media
# e fazer o standard scaler
X_t_scaler

array([[-0.5924806 , -0.50244517],
       [ 0.63878901,  0.78684529],
       [-0.2846632 , -0.48885426],
       ...,
       [ 0.        , -0.17626324],
       [-0.2846632 , -0.04438104],
       [ 0.17706291, -0.49237783]])

## Com Pipeline

Vamos reproduzir as mesmas transformações acima, mas dessa vez vamos usar o Pipeline.

In [19]:
"""
------com_pipeline.py------
Faremos exatamente as mesmas coisas que
fizemos em sem_pipeline.py, mas vamos fazer
uso do Pipeline.

Veja como reduzimos tudo a apenas uma linha, o Pipeline
ja treina e faz as transformacoes para a gente
"""
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

PATH_FILE = 'data/titanic.csv'
USE_COLS = ['PassengerId', 'Survived', 'Age', 'Fare']

df_titanic = pd.read_csv(PATH_FILE, usecols=USE_COLS, index_col='PassengerId')

X = df_titanic.drop(columns='Survived')
y = df_titanic.loc[:, 'Survived']

print('Porcentagem de dados faltantes nas features')
print(f'{(X.isna().mean()*100).round(2)}')
print('Veja que a feature Age tem ~20% de dados faltantes')
print('--------------------------------------------------')

imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()
log_reg = LogisticRegression(random_state=42)

# criando o pipeline com os passos que queremos
pipeline = Pipeline([('ImputeMean', imputer), # primeiro ele insere a media
                     ('StandardScaler', scaler), # segundo ele faz o StandardScaler 
                     ('LogisticRegression', log_reg)]) # terceiro ele treina o estimador

# usamos o .fit() e ele ja faz todos os passos acima
pipeline.fit(X, y)


acc = pipeline.score(X, y)*100
print(f'\nAcc nos mesmos dados que treinamos: {acc:.2f}%')

Porcentagem de dados faltantes nas features
Age     19.87
Fare     0.00
dtype: float64
Veja que a feature Age tem ~20% de dados faltantes
--------------------------------------------------

Acc nos mesmos dados que treinamos: 65.43%


Veja os passos(steps) do Pipeline que criamos.

In [20]:
print('Passos(steps) do Pipeline')
print(pipeline)

Passos(steps) do Pipeline
Pipeline(steps=[('ImputeMean', SimpleImputer()),
                ('StandardScaler', StandardScaler()),
                ('LogisticRegression', LogisticRegression(random_state=42))])


Colocando a configurção display='diagram', conseguimos ver o Pipeline com uma representação em HTML quando usamos Jupyter-Notebook.

In [22]:
from sklearn import set_config
set_config(display='diagram')

pipeline

Pipeline(steps=[('ImputeMean', SimpleImputer()),
                ('StandardScaler', StandardScaler()),
                ('LogisticRegression', LogisticRegression(random_state=42))])

# Dissecando com_pipeline.py